# Test: the impact of an AGN power-law continuum
---
What effect does the inclusion of an AGN power-law continuum have upon the recovered SFH?

In [1]:
%matplotlib widget

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [19]:
import os
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
from itertools import product
import multiprocessing
import pandas as pd

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates, get_bin_edges_and_widths
from ppxftests.mockspec import create_mock_spectrum
from ppxftests.sfhutils import load_sfh, compute_mw_age, compute_lw_age, compute_mass
from ppxftests.sfhutils import compute_mean_age, compute_mean_mass, compute_mean_sfr, compute_mean_1D_sfh
from ppxftests.ppxf_plot import plot_sfh_mass_weighted, plot_sfh_light_weighted

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

fig_path = "/priv/meggs3/u5708159/ppxftests/figs/"
data_path = "/priv/meggs3/u5708159/ppxftests/"


### Defining the AGN continuum
---
In this work, we approximate the AGN continuum as a simple power law,

$$F_{\rm AGN}(\lambda) = F_0 \lambda^{-\alpha_{\lambda}}$$

where the exponent $\alpha_{\lambda} = 2 -\alpha_{\nu}$ where $\alpha_{\nu}$ can range from 1.2 - 2.0 (Groves et al. 2004). 

To determine reasonable values for $F_0$, we use the tight correlation between the total H$\beta$ luminosity, $L_{\rm H\beta}$, and $L_{\rm NT}$, which is defined as the total luminosity of the AGN continuum between 3000 Å and 9500 Å (Neugebauer et al. 1979, Yee 1980), i.e.,

$$L_{\rm NT} = \int^{9500\,Å}_{3000\,Å} F_{\rm AGN}(\lambda) d\lambda 
= \int^{9500\,Å}_{3000\,Å} F_0 \lambda^{-\alpha_{\lambda}} d\lambda 
= \frac{F_0}{1 - \alpha_\lambda}\left(9500^{1 - \alpha_\lambda} - 3000^{1 - \alpha_\lambda} \right)$$

where $F_{\rm AGN}(\lambda)$ is the AGN continuum. In a sample of AGN including quasars, broad & narrow-line radio galaxies and Seyfert 1 & 2 galaxies, Yee (1980) observed a tight correlation such that 

$$L_{\rm NT} \approx 80 L_{\rm H\beta}$$

*Note: equation from [Peterson textbook. eqn. 5.53, p. 90](https://books.google.com.au/books?id=ok4EFlPMStwC&pg=PA90&lpg=PA90&dq=Lnt+agn+continuum&source=bl&ots=QfVvXob4vM&sig=ACfU3U0x69gKrkN-lALkIu0EROAUh1-1vw&hl=en&sa=X&ved=2ahUKEwjW_8fPvqv1AhWXTWwGHarhALcQ6AF6BAgXEAM#v=onepage&q=Lnt%20agn%20continuum&f=false).* 
Referring to table 2 of Dopita et al. (2015), $\log_{10} L_{\rm H\beta} \sim 39 - 42$ in the NLRs of S7 galaxies.
We therefore use our adopted $L_{\rm H\beta}$ in the emission lines to constrain the strength of the AGN continuum in our mock spectra, so that 

$$ F_0 = \frac{80 L_{\rm H\beta}({1 - \alpha_\lambda})}{\left(9500^{1 - \alpha_\lambda} - 3000^{1 - \alpha_\lambda} \right)} $$



In [24]:
###########################################################################
# Defining the AGN continuum using the method of Yee 1980
###########################################################################
L_NT = 1e42

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
for alpha_nu in [0.3, 0.5, 0.7, 1.1, 1.2, 1.3, 1.5, 1.7, 2.0]:
    # Compute the continuum normalisation
    alpha_lambda = 2 - alpha_nu
    L_lambda_0 = L_NT * (1 - alpha_lambda) / (9500**(1 - alpha_lambda) - 3000**(1 - alpha_lambda))

    # Compute the continuum
    lambda_vals_A = np.linspace(3000, 9500, 1e3)
    L_lambda = L_lambda_0 * lambda_vals_A**(-alpha_lambda)

    ax.plot(lambda_vals_A, L_lambda, label=r"$\alpha_\nu =$" + f"{alpha_nu:.1f}")
    ax.set_xlabel("$\lambda$ (Å)")
    ax.set_ylabel("$F(\lambda)$ (erg/s/Å)")
ax.autoscale(axis="x", tight=True, enable=True)
ax.legend()
ax.axvline(3500, color="grey")
ax.axvline(7000, color="grey")
ax.set_title(r"AGN power-law continuum ($L_{\rm NT} = 10^{41}\,\rm erg\,s^{-1}$)")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'AGN power-law continuum ($L_{\\rm NT} = 10^{41}\\,\\rm erg\\,s^{-1}$)')

In [22]:
###########################################################################
# Defining the AGN continuum using the method of Heckman+2005
###########################################################################
L_OIII = 1e40  # erg s^-1 

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
for alpha_nu in [0.3, 0.5, 0.7, 1.1, 1.2, 1.3, 1.5, 1.7, 2.0]:
    # Compute the continuum normalisation
    alpha_lambda = 2 - alpha_nu
    L_5000 = 320 / 5000 * L_OIII  # where L_5000 has units of erg s^-1 Å^-1
    L_lambda_0 = L_5000 / 5000**(-alpha_lambda)

    # Compute the continuum
    lambda_vals_A = np.linspace(3000, 9500, 1e3)
    L_lambda = L_lambda_0 * lambda_vals_A**(-alpha_lambda)

    ax.plot(lambda_vals_A, L_lambda, label=r"$\alpha_\nu =$" + f"{alpha_nu:.1f}")
    ax.set_xlabel("$\lambda$ (Å)")
    ax.set_ylabel("$F(\lambda)$ (erg/s/Å)")
ax.autoscale(axis="x", tight=True, enable=True)
ax.legend()
ax.axvline(3500, color="grey")
ax.axvline(7000, color="grey")
ax.set_title(r"AGN power-law continuum ($L(\rm [O\,III]) = 10^{40}\,\rm erg\,s^{-1}$)")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'AGN power-law continuum ($L(\\rm [O\\,III]) = 10^{40}\\,\\rm erg\\,s^{-1}$)')

In [37]:
plg.close("all")

NameError: name 'plg' is not defined

In [67]:
###########################################################################
# How do these values compare in the S7 sample?
###########################################################################
plt.close("all")

# Load DataFrame containing Hbeta and [O III] fluxes for the S7 sample
df_s7 = pd.read_csv("/home/u5708159/python/Modules/ppxftests/s7_total_line_fluxes.csv")
df_s7 = df_s7.set_index("Unnamed: 0")
gals = df_s7.index.values

# Compute nonthermal continuum parameters for both Yee 1980 and Heckman+2004
df_s7["L_NT (Yee 1980)"] = df_s7["L_HBETA (total)"] * 80
df_s7["L_lambda_5000 (Heckman+2004)"] = df_s7["L_OIII5007 (total)"] * 320 / 5000

# Compute L_lambda_5000 for the Yee 1980 nonthermal continuum, so we can compare directly with Heckman+2004
alpha_nu = 2.0
alpha_lambda = 2 - alpha_nu
F_lambda_0 = df_s7["L_NT (Yee 1980)"] * (1 - alpha_lambda) / (9500**(1 - alpha_lambda) - 3000**(1 - alpha_lambda))
df_s7["L_lambda_5000 (Yee 1980)"] = F_lambda_0 * 5000**(-alpha_lambda)

# Compute logs
df_s7["log L_NT (Yee 1980)"] = np.log10(df_s7["L_NT (Yee 1980)"] )
df_s7["log L_lambda_5000 (Yee 1980)"] = np.log10(df_s7["L_lambda_5000 (Yee 1980)"] )
df_s7["log L_lambda_5000 (Heckman+2004)"] = np.log10(df_s7["L_lambda_5000 (Heckman+2004)"])

In [68]:
# Plot: [OIII] vs. Hbeta
fig, ax = plt.subplots()
ax.scatter(x=df_s7["log L_HBETA (total)"], y=df_s7["log L_OIII5007 (total)"])
ax.plot([0, 50], [0, 50], "k")
ax.set_xlim([37.5, 43])
ax.set_ylim([37.5, 43])
ax.set_xlabel("log L_HBETA (total)")
ax.set_ylabel("log L_OIII5007 (total)")
ax.grid()

# Plot: L_lambda_5000 for both the Yee 1980 and Heckman+2005
fig, ax = plt.subplots()
ax.scatter(x=range(len(gals)), y=df_s7["log L_lambda_5000 (Yee 1980)"].values, label="log L_lambda_5000 (Yee 1980)")
ax.scatter(x=range(len(gals)), y=df_s7["log L_lambda_5000 (Heckman+2004)"].values, label="log L_lambda_5000 (Heckman+2004)")
ax.legend()
ax.grid()
ax.set_xlabel("S7 galaxy #")

# Plot: difference in L_lambda_5000 between the Yee 1980 and Heckman+2005
fig, ax = plt.subplots()
ax.scatter(x=range(len(gals)), y=df_s7["log L_lambda_5000 (Heckman+2004)"].values - df_s7["log L_lambda_5000 (Yee 1980)"].values, label="log L_lambda_5000 (Heckman+2004) - log L_lambda_5000 (Yee 1980)")
ax.axhline(0, color="k")
ax.legend()
ax.grid()
ax.set_xlabel("S7 galaxy #")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'S7 galaxy #')

In [69]:
# Functions for evaluating AGN continua

def agn_cont_Y80(lambda_vals_A, L_Hbeta, alpha_nu):
    alpha_lambda = 2 - alpha_nu
    
    # Compute the continuum normalisation
    L_NT = 80 * L_Hbeta
    L_lambda_0 = L_NT * (1 - alpha_lambda) / (9500**(1 - alpha_lambda) - 3000**(1 - alpha_lambda))

    # Compute the continuum
    return L_lambda_0 * lambda_vals_A**(-alpha_lambda)

def agn_cont_H04(lambda_vals_A, L_OIII, alpha_nu):
    alpha_lambda = 2 - alpha_nu
    
    # Compute the continuum normalisation
    L_5000 = 320 / 5000 * L_OIII  # where L_5000 has units of erg s^-1 Å^-1
    L_lambda_0 = L_5000 / 5000**(-alpha_lambda)

    # Compute the continuum
    return L_lambda_0 * lambda_vals_A**(-alpha_lambda)

In [74]:
gg = 0

In [107]:
# For each S7 galaxy, plot the extracted continuum (in erg s^-1) overlaid with the AGN continuum. 
# This should indicate how reasonable our estimate of the strength of the AGN continuum will be.
fig, ax = plt.subplots(figsize=(12, 5))

data_dir = os.environ["S7_DIR"]
fits_path = os.path.join(data_dir, "4_Nuclear_spectra_Re")

alpha_nu = 2.0

# Open the FITS file containing the extracted spectrum
gal = gals[gg]
hdulist = fits.open(os.path.join(fits_path, f"{gal}_COMB_Re.fits"))
spec = hdulist[0].data
spec_err = hdulist[1].data
lambda_vals_A = np.array(range(hdulist[0].header["NAXIS1"])) + hdulist[0].header["CRVAL1"] * hdulist[0].header["CDELT1"]

# Compute AGN spectra

# Plot
ax.clear()
ax.errorbar(x=lambda_vals_A, y=spec, yerr=spec_err, color="k")
ax.plot(lambda_vals_A, agn_cont_Y80(lambda_vals_A, df_s7.loc[gal, "L_HBETA (total)"], alpha_nu=0.3), label=r"Yee 1980 ($\alpha_\nu = %.1f$)" % 0.3, ls="--", color="fuchsia")
ax.plot(lambda_vals_A, agn_cont_H04(lambda_vals_A, df_s7.loc[gal, "L_OIII5007 (total)"], alpha_nu=0.3), label=r"Heckman+2004 ($\alpha_\nu = %.1f$)" % 0.3, ls="--", color="green")
ax.plot(lambda_vals_A, agn_cont_Y80(lambda_vals_A, df_s7.loc[gal, "L_HBETA (total)"], alpha_nu=2.0), label=r"Yee 1980 ($\alpha_\nu = %.1f$)" % 2.0, ls="-", color="fuchsia")
ax.plot(lambda_vals_A, agn_cont_H04(lambda_vals_A, df_s7.loc[gal, "L_OIII5007 (total)"], alpha_nu=2.0), label=r"Heckman+2004 ($\alpha_\nu = %.1f$)" % 2.0, ls="-", color="green")
ax.grid()
ax.axhline(0, ls="--", color="grey")
ax.autoscale(axis="x", tight=True, enable=True)
ax.set_title(gal)
ax.legend()

gg += 1


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [113]:
# Try reproducing the correlation between [OIII] and "L_B" given in Zakamska+2003
for gal in gals:
    
    # Open the FITS file containing the extracted spectrum
    fname = os.path.join(fits_path, f"{gal}_COMB_Re.fits")
    if os.path.exists(fname):
        hdulist = fits.open(fname)
    else:
        continue
    spec = hdulist[0].data
    spec_err = hdulist[1].data
    dlambda_A = hdulist[0].header["CDELT1"]
    lambda_vals_A = np.array(range(hdulist[0].header["NAXIS1"])) + hdulist[0].header["CRVAL1"] * dlambda_A
    
    # Compute the rest-frame L_B (eqn. 6)
    z = hdulist[0].header["Z"]
    lambda_vals_A /= (1 + z)
    dlambda_A /= (1 + z)
    lambda_start_idx = np.nanargmin(np.abs(lambda_vals_A - 3980))
    lambda_stop_idx = np.nanargmin(np.abs(lambda_vals_A - 4920))
    L_B = np.nansum(spec[lambda_start_idx:lambda_stop_idx] * dlambda_A)
    
    df_s7.loc[gal, "L_B"] = L_B
    df_s7.loc[gal, "log L_B"] = np.log10(L_B)


In [118]:
# plot: is there a strong correlation?
fig, ax = plt.subplots()
ax.scatter(x=df_s7["log L_B"], y=df_s7["log L_OIII5007 (total)"])
fn_Z03 = lambda log_L_B: 1.15 * log_L_B - 3.4
ax.plot([40, 43], [fn_Z03(40), fn_Z03(43)])
ax.set_xlabel("log L_B")
ax.set_ylabel("log L_OIII5007")
ax.grid()
ax.autoscale(tight=True)


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# PPXF: Varying both $L_{\rm NT}$ and $\alpha_\nu$
---

In [4]:
###############################################################################
# Settings
###############################################################################
isochrones = "Padova"
SNR = 100
z = 0.01

# Get the age & metallicity dimensions
_, _, metallicities, ages = load_ssp_templates(isochrones)
N_ages = len(ages)
N_metallicities = len(metallicities)
bin_edges, bin_widths = get_bin_edges_and_widths(isochrones=isochrones)

# ppxf settings
niters = 100
nthreads = 20

# For analysis
lambda_norm_A = 5000
age_thresh_vals = [None, 1e9, None]

# Parameters
# alpha_nu_vals = np.linspace(0.3, 2.1, 5)  # What is a typical value for low-z Seyfert galaxies?
# log_L_NT_vals = np.linspace(42, 44, 5)
alpha_nu_vals = [0.3, 2.0]
log_L_NT_vals = [42, 43, 44]


In [8]:
###########################################################################
# Helper function for running MC simulations
###########################################################################
def ppxf_helper(args):
    # Unpack arguments
    seed, spec, spec_err, lambda_vals_A = args
    
    # Add "extra" noise to the spectrum
    rng = RandomState(seed)
    noise = rng.normal(scale=spec_err)
    spec_noise = spec + noise

    # This is to mitigate the "edge effects" of the convolution with the LSF
    spec_noise[0] = -9999
    spec_noise[-1] = -9999

    # Run ppxf
    pp = run_ppxf(spec=spec_noise, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1,
                  regularisation_method="none", 
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    return pp


In [15]:
###############################################################################
# Load a realistic SFH
###############################################################################
gal = 10
sfh_mw_input, sfh_lw_input, sfr_avg_input, sigma_gas_kms = load_sfh(gal, plotit=True)
M_tot = np.nansum(sfh_mw_input)

# Create the spectrum here, just so we can check it looks OK
create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'sfh_mw_imput' is not defined

In [12]:
###############################################################################
# Before we run ppxf, plot the spectra
###############################################################################
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 4))
ax.set_xlabel("Observed wavelength (Å)")
ax.set_ylabel(r"$F_\lambda(\lambda)\,(erg\,s^{-1}\,Å^{-1})$")

# Plot spectrum without an AGN continuum added as a "control"
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=False)
lambda_norm_idx = np.nanargmin(np.abs(lambda_vals_A - lambda_norm_A))

# Add to plot 
ax.errorbar(x=lambda_vals_A, y=spec, yerr=spec_err, label="No AGN continuum", color="k", zorder=10000, linewidth=0.5)

for alpha_nu, log_L_NT in product(alpha_nu_vals, log_L_NT_vals):
    # Create spectrum
    L_NT = 10**log_L_NT
    spec, spec_err, lambda_vals_A = create_mock_spectrum(
        sfh_mass_weighted=sfh_mw_input,
        agn_continuum=True, L_NT_erg_s=L_NT, alpha_nu=alpha_nu,
        isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
        plotit=False)

    # Add to plot 
    ax.errorbar(x=lambda_vals_A, y=spec, yerr=spec_err, 
                label=r"$\alpha_\nu = %.1f, L_{\rm NT} = 10^{%.1f} \,\rm erg\,s^{-1}$" % (alpha_nu, log_L_NT))

ax.legend(fontsize="x-small")
ax.set_yscale("log")
ax.autoscale(axis="x", tight=True, enable=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Run ppxf with an added AGN continuum to investigate its effect on the recovered light/mass-weighted ages
---

In [16]:
###############################################################################
# Run ppxf without an AGN continuum added as a "control"
###############################################################################
# Create spectrum
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=False)

###########################################################################
# Run ppxf WITHOUT regularisation, using a MC approach
###########################################################################
# Input arguments
seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
args_list = [[s, spec, spec_err, lambda_vals_A] for s in seeds]

# Run in parallel
print(f"Running ppxf on {nthreads} threads...")
t = time()
with multiprocessing.Pool(nthreads) as pool:
    pp_mc_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
print(f"Total time in ppxf: {time() - t:.2f} s")

###########################################################################
# Run ppxf with regularisation
###########################################################################
t = time()
pp_regul = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
              z=z, ngascomponents=1,
              regularisation_method="auto",
              isochrones=isochrones,
              fit_gas=False, tie_balmer=True,
              delta_regul_min=1, regul_max=5e4, delta_delta_chi2_min=1,
              plotit=False, savefigs=False, interactive_mode=False)
print(f"Total time in run_ppxf: {time() - t:.2f} seconds")


Running ppxf on 20 threads...



Total time in ppxf: 173.66 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.84 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.4094...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 55.40 s
Iteration 1: optimal regul = 10000.00; Δm = 2.79519e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 46.040
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 58.34 s
Iteration 2: optimal regul = 20000.00; Δm = 3.00029e+09; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 36.926
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 61.50 s
Iteration 3: optimal regul = 30000.00; Δm = 2.95539e+09; Δregul = 500.00 (Δregul_min

In [34]:
df = pd.DataFrame()

In [35]:
###########################################################################
# Compute mean quantities from the ppxf runs
###########################################################################
thisrow = {}  # Row to append to DataFrame
thisrow["AGN continuum"] = False
thisrow["alpha_nu"] = np.nan
thisrow["log L_NT"] = np.nan

# Compute the mean SFH and SFR from the lists of MC runs
sfh_MC_lw_1D_mean = compute_mean_1D_sfh(pp_mc_list, isochrones, "lw")
sfh_MC_mw_1D_mean = compute_mean_1D_sfh(pp_mc_list, isochrones, "mw")
sfr_avg_MC = compute_mean_sfr(pp_mc_list, isochrones)
sfh_regul_mw_1D = pp_regul.sfh_mw_1D
sfh_regul_lw_1D = pp_regul.sfh_lw_1D
sfr_avg_regul = pp_regul.sfr_mean

# Compute the mean mass- and light-weighted ages plus the total mass in a series of age ranges
for aa in range(len(age_thresh_vals) - 1):
    age_thresh_lower = age_thresh_vals[aa]
    age_thresh_upper = age_thresh_vals[aa + 1]
    
    if age_thresh_lower is None:
        age_thresh_lower = ages[0]
    if age_thresh_upper is None:
        age_thresh_upper = ages[-1]
    age_str = f"{np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f}"
        
    # MC runs: compute the mean mass- and light-weighted ages plus the total mass in this age range
    age_lw_mean, age_lw_std = compute_mean_age(pp_mc_list, isochrones, "lw", age_thresh_lower, age_thresh_upper)
    age_mw_mean, age_mw_std = compute_mean_age(pp_mc_list, isochrones, "mw", age_thresh_lower, age_thresh_upper)
    mass_mean, mass_std = compute_mean_mass(pp_mc_list, isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)
    
    # Regul run: compute the mean mass- and light-weighted ages plus the total mass in this age range
    age_mw_regul = 10**compute_mw_age(sfh_regul_lw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
    age_lw_regul = 10**compute_lw_age(sfh_regul_mw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
    mass_regul = compute_mass(sfh_regul_mw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)

    # Input: compute the mean mass- and light-weighted ages plus the total mass in this age range
    age_mw_input = 10**compute_mw_age(sfh_lw_input, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
    age_lw_input = 10**compute_lw_age(sfh_mw_input, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
    mass_input = compute_mass(sfh_mw_input, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)

    # Put in DataFrame
    thisrow[f"MW age {age_str} (input)"] = age_mw_input
    thisrow[f"LW age {age_str} (input)"] = age_lw_input
    thisrow[f"Mass {age_str} (input)"] = mass_input
    thisrow[f"MW age {age_str} (MC) mean"] = age_mw_mean
    thisrow[f"LW age {age_str} (MC) mean"] = age_lw_mean
    thisrow[f"Mass {age_str} (MC) mean"] = mass_mean
    thisrow[f"MW age {age_str} (MC) std. dev."] = age_mw_std
    thisrow[f"LW age {age_str} (MC) std. dev."] = age_lw_std
    thisrow[f"Mass {age_str} (MC) std. dev."] = mass_std
    thisrow[f"MW age {age_str} (regularised)"] = age_mw_regul
    thisrow[f"LW age {age_str} (regularised)"] = age_lw_regul
    thisrow[f"Mass {age_str} (regularised)"] = mass_regul

df = df.append(thisrow, ignore_index=True)

In [39]:
###############################################################################
# The effect of the strength and exponent of the AGN continuum on the recovered SFH
###############################################################################
for aa, alpha_nu in enumerate(alpha_nu_vals):
    for ll, log_L_NT in enumerate(log_L_NT_vals):
        ###############################################################################
        # Run ppxf without an AGN continuum added as a "control"
        ###############################################################################
        # Create spectrum
        spec, spec_err, lambda_vals_A = create_mock_spectrum(
            sfh_mass_weighted=sfh_mw_input,
            agn_continuum=True, L_NT_erg_s=10**log_L_NT, alpha_nu=alpha_nu,
            isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
            plotit=False)

        ###########################################################################
        # Run ppxf WITHOUT regularisation, using a MC approach
        ###########################################################################
        # Input arguments
        seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
        args_list = [[s, spec, spec_err, lambda_vals_A] for s in seeds]

        # Run in parallel
        print(f"Running ppxf on {nthreads} threads...")
        t = time()
        with multiprocessing.Pool(nthreads) as pool:
            pp_mc_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
        print(f"Total time in ppxf: {time() - t:.2f} s")

        ###########################################################################
        # Run ppxf with regularisation
        ###########################################################################
        t = time()
        pp_regul = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                      z=z, ngascomponents=1,
                      regularisation_method="auto",
                      isochrones=isochrones,
                      fit_gas=False, tie_balmer=True,
                      delta_regul_min=1, regul_max=5e4, delta_delta_chi2_min=1,
                      plotit=False, savefigs=False, interactive_mode=False)
        print(f"Total time in run_ppxf: {time() - t:.2f} seconds")

        ###########################################################################
        # Compute mean quantities from the ppxf runs
        ###########################################################################
        thisrow = {}  # Row to append to DataFrame
        thisrow["AGN continuum"] = True
        thisrow["alpha_nu"] = alpha_nu
        thisrow["log L_NT"] = log_L_NT

        # Compute the mean SFH and SFR from the lists of MC runs
        sfh_MC_lw_1D_mean = compute_mean_1D_sfh(pp_mc_list, isochrones, "lw")
        sfh_MC_mw_1D_mean = compute_mean_1D_sfh(pp_mc_list, isochrones, "mw")
        sfr_avg_MC = compute_mean_sfr(pp_mc_list, isochrones)
        sfh_regul_mw_1D = pp_regul.sfh_mw_1D
        sfh_regul_lw_1D = pp_regul.sfh_lw_1D
        sfr_avg_regul = pp_regul.sfr_mean

        # Compute the mean mass- and light-weighted ages plus the total mass in a series of age ranges
        for aa in range(len(age_thresh_vals) - 1):
            age_thresh_lower = age_thresh_vals[aa]
            age_thresh_upper = age_thresh_vals[aa + 1]

            if age_thresh_lower is None:
                age_thresh_lower = ages[0]
            if age_thresh_upper is None:
                age_thresh_upper = ages[-1]
            age_str = f"{np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f}"

            # MC runs: compute the mean mass- and light-weighted ages plus the total mass in this age range
            age_lw_mean, age_lw_std = compute_mean_age(pp_mc_list, isochrones, "lw", age_thresh_lower, age_thresh_upper)
            age_mw_mean, age_mw_std = compute_mean_age(pp_mc_list, isochrones, "mw", age_thresh_lower, age_thresh_upper)
            mass_mean, mass_std = compute_mean_mass(pp_mc_list, isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)

            # Regul run: compute the mean mass- and light-weighted ages plus the total mass in this age range
            age_mw_regul = 10**compute_mw_age(sfh_regul_lw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
            age_lw_regul = 10**compute_lw_age(sfh_regul_mw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
            mass_regul = compute_mass(sfh_regul_mw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)

            # Input: compute the mean mass- and light-weighted ages plus the total mass in this age range
            age_mw_input = 10**compute_mw_age(sfh_lw_input, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
            age_lw_input = 10**compute_lw_age(sfh_mw_input, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
            mass_input = compute_mass(sfh_mw_input, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)

            # Put in DataFrame
            thisrow[f"MW age {age_str} (input)"] = age_mw_input
            thisrow[f"LW age {age_str} (input)"] = age_lw_input
            thisrow[f"Mass {age_str} (input)"] = mass_input
            thisrow[f"MW age {age_str} (MC) mean"] = age_mw_mean
            thisrow[f"LW age {age_str} (MC) mean"] = age_lw_mean
            thisrow[f"Mass {age_str} (MC) mean"] = mass_mean
            thisrow[f"MW age {age_str} (MC) std. dev."] = age_mw_std
            thisrow[f"LW age {age_str} (MC) std. dev."] = age_lw_std
            thisrow[f"Mass {age_str} (MC) std. dev."] = mass_std
            thisrow[f"MW age {age_str} (regularised)"] = age_mw_regul
            thisrow[f"LW age {age_str} (regularised)"] = age_lw_regul
            thisrow[f"Mass {age_str} (regularised)"] = mass_regul

        df = df.append(thisrow, ignore_index=True)

Running ppxf on 20 threads...



Total time in ppxf: 183.85 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.63 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3931...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 58.96 s
Iteration 1: optimal regul = 10000.00; Δm = 2.63113e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 43.149
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 61.39 s
Iteration 2: optimal regul = 20000.00; Δm = 9.45055e+09; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 23.045
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 62.61 s
Iteration 3: optimal regul = 30000.00; Δm = 5.27867e+09; Δregul = 500.00 (Δregul_min


Total time in ppxf: 175.80 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 3.64 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.4126...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 57.46 s
Iteration 1: optimal regul = 8500.00; Δm = 2.94525e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 1.083
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 63.25 s
Iteration 2: optimal regul = 8400.00; Δm = 9.07374e+07; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 0.357
----------------------------------------------------
STOPPING: Convergence criterion reached; Δχ (goal) - Δχ = 0.35653909864910815; using 8400.00 to produce the best fit
Total time in run_ppxf: 125.76 seconds
Running ppxf on 20 threads...



Total time in ppxf: 265.35 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 5.93 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.5691...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 86.12 s
Iteration 1: optimal regul = 0.00; Δm = 3.05882; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 95.079
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 85.89 s
Iteration 2: optimal regul = 100.00; Δm = 3.63303e+11; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 28.196
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 70.70 s
Iteration 3: optimal regul = 40.00; Δm = 7.85507e+10; Δregul = 20.00 (Δregul_min = 1.00); Δχ (


Total time in ppxf: 193.08 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.83 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3782...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 60.86 s
Iteration 1: optimal regul = 10000.00; Δm = 2.5507e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 51.857
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 65.70 s
Iteration 2: optimal regul = 20000.00; Δm = 5.84765e+09; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 44.825
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 67.86 s
Iteration 3: optimal regul = 30000.00; Δm = 3.52356e+09; Δregul = 500.00 (Δregul_min 


Total time in ppxf: 198.64 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.32 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3922...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 61.53 s
Iteration 1: optimal regul = 10000.00; Δm = 3.2943e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 13.116
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 67.18 s
Iteration 2: optimal regul = 17000.00; Δm = 3.92924e+09; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 0.043
----------------------------------------------------
STOPPING: Convergence criterion reached; Δχ (goal) - Δχ = 0.04252173071087384; using 17000.00 to produce the best fit
Total time in run_ppxf: 134.33 seconds
Running ppxf on 20 threads...



Total time in ppxf: 222.26 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 5.12 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.5864...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 79.00 s
Iteration 1: optimal regul = 500.00; Δm = 4.48507e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 53.032
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 82.78 s
Iteration 2: optimal regul = 100.00; Δm = 6.86619e+10; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 14.372
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 71.38 s
Iteration 3: optimal regul = 60.00; Δm = 3.24681e+10; Δregul = 20.00 (Δregul_min = 1.00)

In [74]:
###############################################################################
# Save DataFrame to file 
###############################################################################
# Add metadata 
df["SNR"] = SNR
df["niters"] = niters
df["nthreads"] = nthreads
df["z"] = z
df["Emission lines"] = False
df["isochrones"] = isochrones
df["sigma_star_kms"] = sigma_star_kms

# Save
df.to_hdf(os.path.join(data_path, f"ga{gal}_agncont.hd5"), key="agn")


### Load the DataFrames storing the output of each AGN continuum run and plot the derived age estimates
---

In [17]:
########################################################
# Plot the recovered mass-weighted mean age vs. alpha_nu
########################################################
pp = PdfPages("/priv/meggs3/u5708159/ppxftests/figs/agn_continuum/agn_cont.pdf")
for gal in range(1, 11):
    df = pd.read_hdf(os.path.join(data_path, f"ga{gal}_agncont.hd5"), key="agn")

    age_thresh_pairs = [
        (ages[0], 1e7),
        (ages[0], 1e8),
        (ages[0], 1e9),
        (1e9, ages[-1]),
        (ages[0], ages[-1]),
    ]

    alpha_nu_vals = [a for a in alpha_nu_vals if ~np.isnan(a)]
    cmap_alpha_nu = lambda aa: matplotlib.cm.get_cmap("Greens", len(alpha_nu_vals) + 1)(aa + 1)

    for age_thresh_pair in [((ages[0], 1e9))]:
        fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(10, 10))
        fig.subplots_adjust(right=0.6)
        fig.subplots_adjust(hspace=0)

        age_thresh_lower, age_thresh_upper = age_thresh_pair
        age_str = f"{np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f}"

        #************************************
        # Plot ages
        for weighttype, ax in zip(["LW", "MW"], axs):

            # Actual value
            ax.axhline(df[f"{weighttype} age {age_str} (input)"].unique()[0], color="gray", label="True value")

            # MC run w/o AGN continuum
            cond = df["AGN continuum"] == False
            ax.axhspan(ymin=df.loc[cond, f"{weighttype} age {age_str} (MC) mean"].values[0] - df.loc[cond, f"{weighttype} age {age_str} (MC) std. dev."].values[0],
                       ymax=df.loc[cond, f"{weighttype} age {age_str} (MC) mean"].values[0] + df.loc[cond, f"{weighttype} age {age_str} (MC) std. dev."].values[0],
                       color="pink", alpha=0.5)
            ax.axhline(df.loc[cond, f"{weighttype} age {age_str} (MC) mean"].values[0], color="pink", label="no AGN continuum")

            # Regul run w/o AGN continuum
            ax.axhline(df.loc[cond, f"{weighttype} age {age_str} (regularised)"].values[0], color="maroon", label="no AGN continuum (regularised)")

            for aa, alpha_nu in enumerate(alpha_nu_vals):
                cond = df["alpha_nu"] == alpha_nu
                # MC
                ax.errorbar(x=df.loc[cond, "log L_NT"].values, 
                            y=df.loc[cond, f"{weighttype} age {age_str} (MC) mean"].values, 
                            yerr=df.loc[cond, f"{weighttype} age {age_str} (MC) std. dev."].values,
                            linestyle="none", marker="D", color=cmap_alpha_nu(aa),
                            label=r"$\alpha_\nu = %.1f$" % alpha_nu, zorder=999)
                # Regularised
                ax.errorbar(x=df.loc[cond, "log L_NT"].values, 
                            y=df.loc[cond, f"{weighttype} age {age_str} (MC) mean"].values, 
                            linestyle="none", marker="o", markerfacecolor="w", color=cmap_alpha_nu(aa),
                            label=r"$\alpha_\nu = %.1f$ (regularised)" % alpha_nu, zorder=999)

            # Decorations
            # ax.set_ylim([1e6, 1e10])
            ax.set_xlabel(r"$\log_{10} L_{\rm NT}$")
            ax.set_ylabel(f"{weighttype}-weighted mean age")
            ax.set_yscale("log")
            ax.grid()

        #************************************
        # Plot mass in each range
        ax = axs[2]

        # Actual value
        M_tot = df["Mass 6.60 < log t < 10.25 (input)"].unique()[0]
        ax.axhline(df[f"Mass {age_str} (input)"].unique()[0] / M_tot, color="gray", label="True value")

        # MC run w/o AGN continuum
        cond = df["AGN continuum"] == False
        ax.axhspan(ymin=(df.loc[cond, f"Mass {age_str} (MC) mean"].values[0] - df.loc[cond, f"Mass {age_str} (MC) std. dev."].values[0]) / M_tot,
                   ymax=(df.loc[cond, f"Mass {age_str} (MC) mean"].values[0] + df.loc[cond, f"Mass {age_str} (MC) std. dev."].values[0]) / M_tot,
                   color="pink", alpha=0.5)
        ax.axhline(df.loc[cond, f"Mass {age_str} (MC) mean"].values[0] / M_tot, color="pink", label="no AGN continuum (MC)")

        # Regul run w/o AGN continuum
        ax.axhline(df.loc[cond, f"Mass {age_str} (regularised)"].values[0] / M_tot, color="maroon", label="no AGN continuum (regularised)")

        # Results from MC runs
        for aa, alpha_nu in enumerate(alpha_nu_vals):
            cond = df["alpha_nu"] == alpha_nu
            # MC
            ax.errorbar(x=df.loc[cond, "log L_NT"].values, 
                        y=df.loc[cond, f"Mass {age_str} (MC) mean"].values / M_tot, 
                        yerr=df.loc[cond, f"Mass {age_str} (MC) std. dev."].values / M_tot,
                        linestyle="none", marker="D", color=cmap_alpha_nu(aa),
                        label=r"$\alpha_\nu = %.1f$" % alpha_nu, zorder=999)
            # Regularised
            ax.errorbar(x=df.loc[cond, "log L_NT"].values, 
                        y=df.loc[cond, f"Mass {age_str} (regularised)"].values / M_tot, 
                        linestyle="none", marker="o", markerfacecolor="w", color=cmap_alpha_nu(aa),
                        label=r"$\alpha_\nu = %.1f$ (regularised)" % alpha_nu, zorder=999)

        # Decorations
        # ax.set_ylim([1e8, 0.5e11])
        ax.set_xlabel(r"$\log_{10} L_{\rm NT}$")
        ax.set_ylabel(r"Mass fraction ($M/M_{\rm tot}$)")
        ax.set_yscale("log")
        ax.grid()

        axs[0].set_title(f"Galaxy ID {gal:004} ({age_str})" + r" $\log_{10}M_{\rm tot} = %.2f$" % np.log10(M_tot))    
        axs[1].legend(bbox_to_anchor=[1.05, 0.5], loc="center left", fontsize="small")
        
    pp.savefig(fig)
    plt.close("all")
    
pp.close()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …